In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import matplotlib.pyplot as plt 
import pickle
import io
import requests
from bs4 import BeautifulSoup

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
## Changing Arbic Characters to Persian Characters !
## Credit to "https://github.com/rezakamalifard/Persian/blob/master/persian/persian.py"
import re
def convert_ar_characters(input_str):
    mapping = {
        'ك': 'ک',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
    }
    return _multiple_replace(mapping, input_str)

def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

# **Loading Balancesheet Data**

In [3]:
# A function to read different file and prepare them
def read_blnc_data(file='98.txt',path=r"C:\Users\Mahdi\OneDrive\Master Thesis\Data"):

    os.chdir(path)
    with open(file,encoding="utf8") as f:
        fileobject = io.StringIO(f.read())

    BlncData = pd.read_csv(fileobject, sep='\t',  lineterminator='\n', names=None)
    
    # Selecting Columns
    BlncData = BlncData[['نماد', 'سال مالی', 'تاریخ مصوب','جمع دارایی‌های جاری',
           'سرمایه گذاری‌ها و سایر دارایی‌ها', 'خالص دارایی‌های ثابت',
           'جمع دارایی‌های غیر جاری', 'جمع کل دارایی‌ها', 'جمع بدهی‌های جاری',
           'جمع بدهی‌های غیر جاری', 'جمع کل بدهی‌ها', 'سرمایه',
           'سود و زیان انباشته', 'اندوخته قانونی',
           'جمع حقوق صاحبان سهام در پایان سال مالی',
           'جمع کل بدهی‌ها و حقوق صاحبان سهام',
           'جمع حقوق صاحبان سهام مصوب (در مجمع عادی)']]
    
    # renaming columns
    BlncData.rename(columns={'نماد':'Symbol','سال مالی':'Fin_year','جمع دارایی‌های جاری':'Tot_current_asset','تاریخ مصوب':'approve_date',
                             'خالص دارایی‌های ثابت':'Net_fixed_assed','سرمایه گذاری‌ها و سایر دارایی‌ها':'other_asset',
                             'جمع بدهی‌های جاری':'Tot_current_lib','جمع کل دارایی‌ها':'Tot_asset','جمع دارایی‌های غیر جاری':'Tot_uncurrent_asset',
                             'سرمایه':'Capital','حقوق عمومی':'Public_rights','جمع کل بدهی‌ها':'Tot_lib','جمع بدهی‌های غیر جاری':'Tot_uncurrent_lib',
                             'سایر اندوخته‌ها':'Other_saving','اندوخته قانونی':'Reserved_saving','سود و زیان انباشته':'Comulated_profit_loss',
                             'جمع حقوق صاحبان سهام در پایان سال مالی':'Equity_at_year_end','جمع کل بدهی‌ها و حقوق صاحبان سهام':'Debt_Equity',
                              'جمع حقوق صاحبان سهام مصوب (در مجمع عادی)':'Debt_Equity_normal'},inplace=True)

    # DataOrg.Symbol: convert_ar_characters(x)
    Names = BlncData.Symbol.drop_duplicates()
    Conv_Names = Names.apply(lambda x : convert_ar_characters(x))
    BlncData_Symbol_ArtoFa_dict = dict(zip(Names,Conv_Names))
    BlncData['Symbol'] = BlncData.Symbol.map(BlncData_Symbol_ArtoFa_dict)

    # Dates
    BlncData = BlncData[~pd.isnull(BlncData.Fin_year)]
    BlncData.Fin_year = BlncData.Fin_year.apply(lambda x: jd.date(day=int(x[8:10]), month=int(x[5:7]),year=int(x[0:4])))

    BlncData = BlncData[~pd.isnull(BlncData.approve_date)]
    BlncData.approve_date = BlncData.approve_date.apply(lambda x: jd.date(day=int(x[8:10]), month=int(x[5:7]),year=int(x[0:4])))
    
    # changing to int
    for x in BlncData.columns[3:]:
        BlncData = BlncData[~pd.isnull(BlncData[x])]
        BlncData[x] = BlncData[x].apply(lambda x: int(x.replace(',','')))
        
    return(BlncData)

In [4]:
blnc_data = read_blnc_data(file='98.txt')
blnc_data['book_value'] = blnc_data.Tot_asset-blnc_data.Tot_lib
blnc_data = blnc_data[blnc_data.book_value>0]

# **Loading Shareholder Data**

In [5]:
## Loading DATA
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
SDATA = pd.read_csv("Shareholder97.csv",index_col=0)

# Conver date from string to jdatetime
SDATA['True_Date'] = pd.to_datetime(SDATA['True_Date'], format='%Y-%m-%d')
G = SDATA.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
DataOrg_date_GtoJ_dict = dict(zip(G,J))
SDATA['Jalali_Date']=SDATA.True_Date.map(DataOrg_date_GtoJ_dict)

SDATA.drop(columns=['High', 'Low', 'Open', 'Last', 'Volume', 'close',
       'True_Date', 'year', 'month', 'day', 'Fill_Flag','Unadjusted_close','chnk_id'],inplace=True)

In [6]:
# Filtering and keeping symbols that we have data in both datasets
sym_list = list(set(blnc_data.Symbol).intersection(set(SDATA.Symbol.drop_duplicates())))
SDATA = SDATA[SDATA.Symbol.isin(sym_list)]

In [7]:
# Creating Dataframe for saving concentration mearsurs
CMdf = SDATA.groupby('Symbol',as_index=False).agg({'Id_tse':'first','percent':'sum','ShareHolder':'count'}).rename(columns={'ShareHolder':'Num_holders','percent':'sum_over1'})
CMdf.reset_index(drop=True,inplace=True)

# **Concentration Measures**

## 1- Largest Owner

In [8]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':'max'}).rename(columns={'percent':'Largest_Owner'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left')

## 2- First/Second

In [9]:
def nth_max(data,nth=1,interval=False):
    data = data.sort_values(ascending=False)
    if interval:
        return(np.round(data.iloc[min(nth[0]-1,len(data)-1):min(nth[1],len(data))],2))
    else:
        return(np.round(data.iloc[min(nth-1,len(data)-1)],2))

In [10]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/nth_max(x,nth=2,interval=False)}}).rename(columns={'percent':'First_Second'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Second', '<lambda>'):'First_Second'})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 3- First/Sumtwofour

In [11]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/sum(nth_max(x,nth=[2,4],interval=True))}}).rename(columns={'percent':'First_Sumtwofour'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Sumtwofour', '<lambda>'):'First_Sumtwofour'})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 4- Sumfive

In [12]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum(nth_max(x,nth=[1,5],interval=True))/100}}).rename(columns={'percent':'Sumfive'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Sumfive', '<lambda>'):'Sumfive'})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 5- Gini

We calculate gini coeficient using Deaton 1997:
$$ \gamma = \frac{N+1}{N-1} - \frac{2}{\mu\times N\times(N-1)}\sum_{i=1}^N{\rho_ix_i}$$

In [13]:
def gini(data):
    data.sort(reverse = True)
    N = len(data)
    mu = np.mean(data)
    ser = [(i+1)*data[i] for i in range(len(data))]
    try:
        gamma = (N+1)/(N-1)-(2*sum(ser))/(mu*N*(N-1))
    except:
        gamma = 0
    return(gamma)

In [14]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: gini(list(x))}}).rename(columns={'percent':'Gini'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Gini', '<lambda>'):'Gini'})

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 6- Herfindhal

In [15]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum([(t/100)**2 for t in list(x)])}}).rename(columns={'percent':'Herfindhal'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Herfindhal', '<lambda>'):'Herfindhal'})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 7-Shapley-Shubik

## 8-Banzhaf

In [16]:
#####################################################################

In [221]:
df = SDATA[SDATA.Symbol==sym_list[0]]
percent = list(df.percent)
percent.sort(reverse=True)

prc = [x for x in percent]

prc_str = ''
for x in prc:
    prc_str+=str(x)
    prc_str+=' '



url = "https://mywebpages.csv.warwick.ac.uk/cgi-vpi/ssdirect.cgi"


payload = {'numberofplayers': len(df),
           'quota': 50.01,
           'textarea': prc_str}

response = requests.request("POST", url, data = payload)

    
html_out = response.text.encode('utf8')
print('It took about ',np.round(response.elapsed.microseconds/1000000,2), 'seconds')

parsed_html = BeautifulSoup(html_out)
rows = parsed_html('tr')

data = []
for row in rows:
    if row.th:
        cols = row.find_all('th')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    else:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    
    
out = pd.DataFrame(data[1:], columns=data[0])
out.iloc[:,0] = prc
out


It took about  0.64 seconds


,WEIGHT,SHAPLEY-SHUBIK
0,50.25,1.000000
1,16.66,0.000000
2,15.77,0.000000
3,2.28,0.000000


In [230]:
# URL Dict
URL={'direct':"https://mywebpages.csv.warwick.ac.uk/cgi-vpi/ssdirect.cgi",
     'genf':"https://mywebpages.csv.warwick.ac.uk/cgi-vpi/ssgenf.cgi",
     'mmle':"https://mywebpages.csv.warwick.ac.uk/cgi-vpi/ssmmle.cgi",
     'ocean':"https://mywebpages.csv.warwick.ac.uk/cgi-vpi/ssocean.cgi"}

def find_shapley(Symbol,how='direct',quota = 50.01):
    
    # Finding data of Symbol
    df = SDATA[SDATA.Symbol==Symbol]
    percent = list(df.percent)
    percent.sort(reverse=True)
    
    
    if len(df)==1:
        return('Length Error')
    
    url = URL[how]
    
    if how=='direct':
        prc = percent
        prc_str = ''
        for x in prc:
            prc_str+=str(x)
            prc_str+=' '
            
        payload = {'numberofplayers': len(df),
                   'quota': quota,
                   'textarea': prc_str}
        
    elif how=='genf':
        prc = [int(x*100)for x in percent]
        prc_str = ''
        for x in prc:
            prc_str+=str(x)
            prc_str+=' '
            
        payload = {'numberofplayers': len(df),
                   'quota': int(quota*100),
                   'textarea': prc_str}
        
    
    
    try:
        response = requests.request("POST", url, data = payload)
    except:
        return('Error:'+str(response.status_code))
    
    html_out = response.text.encode('utf8')
    print('It took about ',np.round(response.elapsed.microseconds/1000000,2), 'seconds')
    
    parsed_html = BeautifulSoup(html_out)
    rows = parsed_html('tr')

    data = []
    for row in rows:
        if row.th:
            cols = row.find_all('th')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values
        else:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele]) # Get rid of empty values
    
    try:
        out = pd.DataFrame(data[1:], columns=data[0])
        out.iloc[:,0] = prc
        return(out)
    except:
        print('DataFrame Error')
        return(data)

In [231]:
find_shapley(sym_list[1],how='direct',quota = 50.01)

'Length Error'